In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#           Machine Learning y Deep Learning
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#         Aprendizaje con datos desbalanceados: 
#  algoritmos de sobremuestreo, submuestreo, SMOTE y ADASYN
# ==========================================================
import numpy as np
import pandas as pd
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [ ]:
# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/mortalidad_cochabamba.csv'

# Cargar los datos en un DataFrame
df_mortalidad = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(df_mortalidad.head())

# Preprocesamiento y preparacion de datos:
# Dropear la categoría "Subita" de la columna "causa_muertes"
df_mortalidad = df_mortalidad[df_mortalidad['causa_muertes'] != 'Subita']

# Crear la variable target: 1 si "causa_muertes" es "Suicidio", 0 si es "Natural"
df_mortalidad['target'] = df_mortalidad['causa_muertes'].apply(lambda x: 1 if x == 'Suicidio' else 0)

# Seleccionar las columnas numéricas y convertir las categóricas a variables dummies
numerical_cols = ['edad']
categorical_cols = ['depto_nacimiento', 'lugar_de_fallecimiento', 'estado_civil', 
                    'grado_instruccion', 'tuvo_atencion_medica', 'atendio_el_medico_que_suscribe']

# Convertir las variables categóricas a dummies
df_categorical_dummies = pd.get_dummies(df_mortalidad[categorical_cols], drop_first=True)

# Crear la matriz X con las variables numéricas y categóricas recodificadas binarias
X = pd.concat([df_mortalidad[numerical_cols], df_categorical_dummies], axis=1)

# Mostrar las primeras filas de la matriz X y la variable target
X.head(), df_mortalidad['target'].head()

# Definir la variable target
y = df_mortalidad['target']

# Verificar el balance de la variable target
balance_target = df_mortalidad['target'].value_counts(normalize=True)
balance_target

In [ ]:
# Aplicar SMOTE para generar nuevas muestras sintéticas
semilla = 
smote = SMOTE(random_state=semilla)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Separar los datos originales y los resampleados por clase
X_minority = X.loc[y == 0]  
X_majority = X.loc[y == 1]  
X_minority_resampled = X_resampled[y_resampled == 0] 

# Visualizar los datos originales y los sintéticos generados por SMOTE
plt.figure(figsize=(8, 6))

# Datos originales
plt.scatter(X_majority.iloc[:, 0], X_majority.iloc[:, 1], 
            label='Clase mayoritaria muerte natural (datos reales)', alpha=0.5)  
plt.scatter(X_minority.iloc[:, 0], X_minority.iloc[:, 1], 
            label='Clase minoritaria suicidios (datos reales)', color='red', alpha=0.7)  

# Nuevos datos generados por SMOTE
plt.scatter(X_minority_resampled.iloc[:, 0], 
            X_minority_resampled.iloc[:, 1], 
            label='Clase minoritaria suicidios (datos sintéticos)', color='orange', alpha=0.6, marker='x')

# Configurar etiquetas y título
plt.xlabel('edad')
plt.ylabel('nacio en Cochabamba')
plt.title('SMOTE en casos desbalanceados')
plt.legend()
plt.grid(True)
plt.show()

# Crear una tabla de frecuencia para la clase objetivo
frequency_table = pd.Series(y_resampled).value_counts().sort_index()
frequency_table.index = ['Clase mayoritaria (muerte natural):', 'Clase minoritaria (suicidios):']

# Mostrar la tabla de frecuencia
print("Tabla de Frecuencia de la Clase Objetivo después de SMOTE:")
print(frequency_table)

In [ ]:
# Crear el pipeline que aplica SMOTE y luego entrenr un modelo XGBoost
# Definir el modelo de XGBoost
semilla = 
modelo = xgb.XGBClassifier(random_state=semilla)

pipeline = Pipeline([
    ('smote', SMOTE(random_state=semilla)),
    ('model', modelo)
])

# Validación cruzada k-fold estratificada
particiones_k = 9
skf = StratifiedKFold(n_splits=particiones_k, shuffle=True, random_state=semilla)

# Definir la métrica de AUC
auc_scorer = make_scorer(roc_auc_score)

# Realizar la validación cruzada k-fold estratificada y calcular el AUC para cada fold
cv_auc_scores = cross_val_score(pipeline, X, y, cv=skf, scoring=auc_scorer)

# Mostrar los resultados de AUC para cada fold y la media
print(f'AUC para cada fold: {cv_auc_scores}')
print(f'AUC promedio: {cv_auc_scores.mean()}')

# Crear un gráfico de barras con los resultados de AUC en cada fold
mean_auc = cv_auc_scores.mean()
folds = np.arange(1, len(cv_auc_scores) + 1)

plt.figure(figsize=(4, 4))
plt.bar(folds, np.array(cv_auc_scores), color='skyblue', edgecolor='black')
plt.axhline(y=mean_auc, color='r', linestyle='--', label=f'AUC promedio = {mean_auc:.2f}')

# Configurar etiquetas y título
plt.xlabel('Fold')
plt.ylabel('AUC')
plt.title('AUC: Validación cruzada k-fold estratificada y SMOTE')
plt.xticks(folds)
plt.ylim(0.2, 1.0)  # Ajustar los límites de y para una mejor visualización
plt.legend(loc='lower right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Definir las técnicas de balanceo
semilla = 
tecnicas_balanceo = {
    'SMOTE': SMOTE(random_state=semilla),
    'ADASYN': ADASYN(random_state=semilla),
    'RandomOverSampler': RandomOverSampler(random_state=semilla),
    'RandomUnderSampler': RandomUnderSampler(random_state=semilla)
}

# Validación cruzada k-fold estratificada
particiones_k =  # particiones de datos 
skf = StratifiedKFold(n_splits=particiones_k, shuffle=True, random_state=semilla)

# Definir la métrica de AUC
auc_scorer = make_scorer(roc_auc_score)

# Evaluar cada técnica de balanceo
resultados = {}
for nombre, tecnica in tecnicas_balanceo.items():
    pipeline = Pipeline([
        ('balanceo', tecnica),
        ('modelo', modelo)
    ])
    
    cv_auc_scores = cross_val_score(pipeline, X, y, cv=skf, scoring=auc_scorer)
    resultados[nombre] = cv_auc_scores
    print(f'{nombre} - AUC promedio: {cv_auc_scores.mean()}')

In [ ]:
# Visualizar los resultados comparativos
mean_aucs = {nombre: np.mean(auc_scores) for nombre, auc_scores in resultados.items()}
std_aucs = {nombre: np.std(auc_scores) for nombre, auc_scores in resultados.items()}

# Crear un boxplot para comparar los resultados de AUC de cada técnica
plt.figure(figsize=(10, 6))
plt.boxplot([resultados[nombre] for nombre in tecnicas_balanceo.keys()], 
            labels=tecnicas_balanceo.keys(), patch_artist=True)
plt.ylabel('AUC')
plt.title('Distribución de AUC por Técnica de Balanceo de Clases')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(0, 1)  # Ajustar los límites del eje y para mejor visualización
plt.show()